In [1]:
# !pip install torchaudio==0.9.0
import torch
import torchaudio
import numpy as np
from tqdm import tqdm
import torch.nn as nn

## 0. Download dataset
- `wget` is a linux command to download a file from web

In [3]:
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.001
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.002
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.003

--2021-10-29 11:27:11--  http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.001
Resolving mi.soi.city.ac.uk (mi.soi.city.ac.uk)... 138.40.249.126
Connecting to mi.soi.city.ac.uk (mi.soi.city.ac.uk)|138.40.249.126|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.001 [following]
--2021-10-29 11:27:12--  https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.001
Resolving mirg.city.ac.uk (mirg.city.ac.uk)... 138.40.249.131
Connecting to mirg.city.ac.uk (mirg.city.ac.uk)|138.40.249.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1100000000 (1.0G) [application/zip]
Saving to: 'mp3.zip.001.1'

mp3.zip.001.1         9%[>                   ]  96.76M   174KB/s    eta 75m 2s ^C
--2021-10-29 11:34:51--  http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.002
Resolving mi.soi.city.ac.uk (mi.soi.city.ac.uk)... 138.40.249.126
Connecting to mi.soi.city.ac.uk (mi.soi.c

- Concatenate the zip files and unzip it

In [ ]:
!cat mp3.zip.* > mp3_all.zip
!unzip -q mp3_all.zip -d MTAT/

- Download meta data

In [ ]:
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/annotations_final.csv -P MTAT/
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/clip_info_final.csv -P MTAT/


--2021-10-11 06:32:10--  http://mi.soi.city.ac.uk/datasets/magnatagatune/annotations_final.csv
Resolving mi.soi.city.ac.uk (mi.soi.city.ac.uk)... 138.40.249.126
Connecting to mi.soi.city.ac.uk (mi.soi.city.ac.uk)|138.40.249.126|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://mirg.city.ac.uk/datasets/magnatagatune/annotations_final.csv [following]
--2021-10-11 06:32:11--  https://mirg.city.ac.uk/datasets/magnatagatune/annotations_final.csv
Resolving mirg.city.ac.uk (mirg.city.ac.uk)... 138.40.249.131
Connecting to mirg.city.ac.uk (mirg.city.ac.uk)|138.40.249.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21517373 (21M) [text/csv]
Saving to: ‘mtat_dataset/annotations_final.csv’

annotations_final.c 100%[===================>]  20.52M  6.25MB/s    in 3.3s    

2021-10-11 06:32:16 (6.25 MB/s) - ‘mtat_dataset/annotations_final.csv’ saved [21517373/21517373]

--2021-10-11 06:32:16--  http://mi.soi.city.ac.uk/data

## 1. Look at the Dataset

In [4]:
import pandas as pd
from pathlib import Path

data_dir = Path('../MTAT/')

In [4]:
meta = pd.read_csv(data_dir / "clip_info_final.csv", delimiter='\t')
labels = pd.read_csv(data_dir / "annotations_final.csv", delimiter='\t')

In [15]:
meta.head()

,clip_id,track_number,title,artist,album,url,segmentStart,segmentEnd,original_url,mp3_path
0,2,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,30,59,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,146,175,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,262,291,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,291,320,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,320,349,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...


In [16]:
labels.head()

,clip_id,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...


In [6]:
tag = [key for key in labels.keys() if not key in ('clip_id', 'mp3_path')]
len(tag), tag[:10]

(188,
 ['no voice',
  'singer',
  'duet',
  'plucking',
  'hard rock',
  'world',
  'bongos',
  'harpsichord',
  'female singing',
  'clasical'])

- Some of the tags have different name but represents almost same things
- Below are the example of defining synonyms among tags

In [2]:
# From https://github.com/keunwoochoi/magnatagatune-list
synonyms = [['beat', 'beats'],
			['chant', 'chanting'],
			['choir', 'choral'],
			['classical', 'clasical', 'classic'],
			['drum', 'drums'],
			['electro', 'electronic', 'electronica', 'electric'],
			['fast', 'fast beat', 'quick'],
			['female', 'female singer', 'female singing', 'female vocals', 'female voice', 'woman', 'woman singing', 'women'],
			['flute', 'flutes'],
			['guitar', 'guitars'],
			['hard', 'hard rock'],
			['harpsichord', 'harpsicord'],
			['heavy', 'heavy metal', 'metal'],
			['horn', 'horns'],
			['india', 'indian'],
			['jazz', 'jazzy'],
			['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
			['no beat', 'no drums'],
			['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
			['opera', 'operatic'],
			['orchestra', 'orchestral'],
			['quiet', 'silence'],
			['singer', 'singing'],
			['space', 'spacey'],
			['string', 'strings'],
			['synth', 'synthesizer'],
			['violin', 'violins'],
			['vocal', 'vocals', 'voice', 'voices'],
			['strange', 'weird']]

In [17]:
labels = pd.read_csv(data_dir / "annotations_final.csv", delimiter='\t')
for syn in synonyms:
  for i in range(1, len(syn)):
    labels[syn[0]] += labels[syn[1]]
  labels = labels.drop(columns=syn[1:])
  labels[syn[0]] = np.minimum(labels['beat'].values, 1)
tag = [key for key in labels.keys() if not key in ('clip_id', 'mp3_path')]
len(tag), tag[:10]
labels

,clip_id,singer,duet,plucking,world,bongos,harpsichord,sitar,chorus,female opera,...,soft,noise,choir,rap,hip hop,water,baroque,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25858,58899,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-56-la_bres...
25859,58906,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...
25860,58907,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...
25861,58908,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...


In [5]:
class MTATDataset:
  def __init__(self, dir, top_k=50, min_tag_occur=1, synonyms=(), max_num_data=None, sr=16000, split='train'):
    self.dir = dir
    self.meta = pd.read_csv(data_dir / "clip_info_final.csv", delimiter='\t')
    self.labels = pd.read_csv(data_dir / "annotations_final.csv", delimiter='\t')
    self.sr = sr

    self.get_tag_names()
    self.combine_tags(synonyms)
    self.filter_top_k_tags(top_k)
    self.filter_minimum_tag(min_tag_occur)


    if split=="train":
      sub_dir_ids = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c']
    elif split=='valid':
      sub_dir_ids = ['d']
    else: #test
      sub_dir_ids = ['e', 'f', 'g']

    is_in_set = [True if x[0] in sub_dir_ids else False for x in self.labels['mp3_path'].values.astype('str')]
    self.labels = self.labels.iloc[is_in_set]
    if max_num_data:
      self.labels = self.labels.iloc[:max_num_data]

    self.vocab = self.labels.columns.values[1:-1]
    # self.audios = self.load_audio()
    self.audios = self.load_pre_processed_data()
    self.label_tensor = self.convert_label_to_tensor()

  def get_tag_names(self):
    self.tag_names = [key for key in self.labels.keys() if not key in ('clip_id', 'mp3_path')]

  def combine_tags(self, synonyms):
    for syn in synonyms:
      for i in range(1, len(syn)):
        self.labels[syn[0]] += self.labels[syn[i]]
      self.labels = self.labels.drop(columns=syn[1:])
      self.labels[syn[0]] = np.minimum(self.labels[syn[0]].values, 1)
    self.get_tag_names()

  def clip_id_to_tag(self, clip_id):
    return torch.Tensor(self.labels[self.labels['clip_id']==clip_id].values[0,1:-1].astype(int))

  def filter_top_k_tags(self, k=50):
    tag_count = [sum(self.labels[tag]) for tag in self.tag_names]
    descending_sorted_idx = sorted(range(len(tag_count)), key=lambda k: -tag_count[k])
    filtered_tag_names = np.asarray(self.tag_names)[descending_sorted_idx[:k]].tolist()
    for tag in self.tag_names:
      if tag not in filtered_tag_names:
        self.labels = self.labels.drop(columns=tag)
    self.tag_names = filtered_tag_names
  
  def filter_minimum_tag(self, min_tag=1):
    tag_occurs_by_song = np.sum(self.labels[self.tag_names].values, axis=1)
    self.labels = self.labels.drop(index=np.where(tag_occurs_by_song < min_tag)[0])

  def load_audio(self):
    audios = []
    fail_idx = []
    for idx in tqdm(range(len(self.labels))):
      try:
        audio, sr = torchaudio.load(self.dir / self.labels.iloc[idx]['mp3_path'])
        if sr != self.sr:
          audio = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sr)(audio)
        audios.append(audio)
      # try
      except:
        fail_idx.append(idx)
    self.labels = self.labels.drop(index=fail_idx)
    return audios

  def load_pre_processed_data(self):
    audios = [torch.load( (self.dir / self.labels.iloc[idx]['mp3_path']).with_suffix('.pt'))['audio'][0] for idx in tqdm(range(len(self.labels)))]
    return audios

  def convert_label_to_tensor(self):
    return torch.LongTensor(self.labels.values[:, 1:-1].astype('bool'))

  def __getitem__(self, idx):
    audio = self.audios[idx]
    tag = self.label_tensor[idx]
    return audio, tag

  def __len__(self):
    return len(self.labels)


dataset = MTATDataset(data_dir, synonyms=synonyms, max_num_data=5000)

100%|██████████| 5000/5000 [00:03<00:00, 1314.26it/s]


In [6]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [20]:
torch.Tensor([0]).cuda()

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [7]:
batch = next(iter(train_loader))
batch

[tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           1.7881e-07,  5.3644e-07],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -3.5763e-07,
           8.3447e-07, -2.1458e-06],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  9.5367e-07,
          -5.9605e-07, -6.5565e-07],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.7881e-07,
          -1.7881e-07, -2.3842e-07],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -5.9605e-07,
          -6.5565e-07,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.9670e-06,
          -1.6689e-06,  3.5763e-07]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])]

## Make model

In [8]:
class AudioSpecModel(nn.Module):
  def __init__(self, sr, n_fft, hop_length, n_mels) -> None:
    super().__init__()
    self.mel_converter = torchaudio.transforms.MelSpectrogram(sample_rate=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    self.db_converter = torchaudio.transforms.AmplitudeToDB()
  
  def forward(self, x):
    mel_spec = self.mel_converter(x)
    return self.db_converter(mel_spec)

class AudioModel(nn.Module):
  def __init__(self, sr, n_fft, hop_length, n_mels, num_output, hidden_size) -> None:
    super().__init__()
    self.mel = AudioSpecModel(sr, n_fft, hop_length, n_mels)
    self.conv_layer = nn.Sequential(
      nn.Conv1d(n_mels, out_channels=hidden_size, kernel_size=3),
      nn.MaxPool1d(3),
      nn.ReLU(),
      nn.Conv1d(hidden_size, out_channels=hidden_size, kernel_size=3),
      nn.MaxPool1d(3),
      nn.ReLU(),
      nn.Conv1d(hidden_size, out_channels=hidden_size, kernel_size=3),
      nn.MaxPool1d(3),
      nn.ReLU()
    )
    self.final_layer = nn.Linear(hidden_size, num_output)

  def forward(self, x):
    spec = self.mel(x) # N X MEL X L (Time)
    conv_output = self.conv_layer(spec) # N X OutChannel X L_out
    max_output = torch.max(conv_output, dim=2)[0]
    linear_out = self.final_layer(max_output)
    
    return linear_out.sigmoid()


model = AudioModel(sr=16000, n_fft=1024, hop_length=512, n_mels=48, num_output=50, hidden_size=32)
spec_converter = AudioSpecModel(sr=16000, n_fft=1024, hop_length=512, n_mels=48)


In [9]:
def train_model(model, train_loader, valid_loader, optimizer, num_epochs, loss_func, device='cuda'):
  loss_records =[] 
  model.train()
  for epoch in range(num_epochs):
    for batch in train_loader:
      optimizer.zero_grad()
      audio, label = batch
      pred = model(audio.to(device))
      loss = loss_func(pred, label.to(device))
      loss.backward()
      optimizer.step()
      loss_records.append(loss.item())
    # valid_model()
  return loss_records

def valid_model(model, valid_loader, device):
  for batch in valid_loader:
    audio, label = batch
    # pred 


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model = model.to('cuda')
loss_func = torch.nn.BCELoss()
loss = train_model(model, train_loader, None, optimizer, 10, loss_func )

/home/teo/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA RTX A5000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA RTX A5000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [85]:
!nvcc --version

/bin/bash: nvcc: command not found


In [62]:
torch.set_printoptions(sci_mode=False)

In [86]:
batch[0].to('cuda')

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [78]:
torch.max(spec_converter(batch[0]), dim=1)

torch.return_types.max(
values=tensor([[-40.7118, -21.5218,  21.8099,  ...,  16.2576,  -3.4054, -85.3622],
        [-26.3913,  -7.5810,  25.9024,  ...,  24.5974,   3.7532, -69.6471],
        [-37.8599, -20.1594,  24.7856,  ...,  27.4672,   3.7065, -75.1246],
        ...,
        [-40.7829, -19.1795,  26.4167,  ...,  34.2211,  15.7194, -54.3557],
        [-28.5454,  -5.7235,  20.7069,  ...,  28.2244,   9.7189, -59.2974],
        [-54.0965, -34.0383,   9.5253,  ...,  22.8983,   5.0197, -81.2673]]),
indices=tensor([[10, 11,  6,  ...,  6,  6, 13],
        [22, 22, 21,  ..., 14,  6, 19],
        [ 5,  4,  7,  ...,  9,  9, 10],
        ...,
        [18, 26,  4,  ...,  7,  8,  9],
        [ 5,  5, 10,  ..., 12, 12,  9],
        [ 7, 10,  9,  ...,  6, 10, 21]]))

In [77]:
model(batch[0])

TypeError: linear(): argument 'input' (position 1) must be Tensor, not torch.return_types.max

In [ ]:
from collections import Counter
clip_counter = Counter(labels['clip_id'])

In [8]:
id_n_path = meta[['clip_id', 'mp3_path']].to_dict('records')
pt_files = list(data_dir.rglob('*.pt'))

25860

In [ ]:
id_n_path[0]['mp3_path']

'f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-30-59.mp3'

In [ ]:
mp3_path.exists()

True

In [10]:
def resample_files(id_n_path:list, data_dir:Path, labels, SR:int):
  SR = 16000
  for data_info in id_n_path:
    if not isinstance(data_info['mp3_path'], str):
      continue
    mp3_path = data_dir / data_info['mp3_path']
    if mp3_path.exists():
      try:
        y, sr = torchaudio.load(mp3_path)
      except:
        print(f'Failed to load {str(mp3_path)}')
        continue
      if sr != SR:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SR)
        y = resampler(y)
      label = labels[labels['clip_id']==data_info['clip_id']].values[0,1:-1].astype(int)
      torch.save({'audio':y, 'tag':torch.LongTensor(label)}, mp3_path.with_suffix('.pt'))

id_n_path = meta[['clip_id', 'mp3_path']].to_dict('records')


Failed to load ../MTAT/6/norine_braun-now_and_zen-08-gently-117-146.mp3
Failed to load ../MTAT/8/jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3
Failed to load ../MTAT/9/american_baroque-dances_and_suites_of_rameau_and_couperin-25-le_petit_rien_xiveme_ordre_couperin-88-117.mp3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.mkdir('mtat_dataset')

In [ ]:
for path in Path('.').glob('*'):
  if path in ['mtat_dataset', 'drive']:
    continue
  if path.is_dir() and len(str(path))==1:
    os.rename(path, 'mtat_dataset/'+str(path))

In [ ]:
!zip -r mtat.zip mtat_dataset

In [ ]:
import os
for pt_file in Path('mtat_dataset').rglob('*.pt'):
  os.rename(pt_file, 'mtat_pt/'+pt_file.name)

In [ ]:
test = next(Path('mtat_pt').rglob('*.pt'))

In [ ]:
test.name

'lvx_nova-lvx_nova-05-my_time-552-581.pt'

In [ ]:
!rm -rf mtat_pt/